# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this project.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

The next step is to import the constituents of the S&P 500.

These constituents change over time, so in an ideal world we would connect directly to the index provider and pull their real-time constituents on a regular basis.

The index provider for real-time data are paid, so here we are using a static version which is free. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). This file is moved into the same folder where this python file exists.

Now it's time to import these stocks to the Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import the IEX Cloud API token. This is the data provider that we will use in this project.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to the local Git repository. We will use a sandbox API token in this project, which means that the data we will use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download the `secrets.py` file. The file is moved into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'EQLBMASLS(TSDE GO AGANNKE RT)/CAL',
 'calculationPrice': 'close',
 'open': 148.7,
 'openTime': 1666363507901,
 'openSource': 'iacffilo',
 'close': 153.35,
 'closeTime': 1645219754001,
 'closeSource': 'ilicofaf',
 'high': 153.35,
 'highTime': 1657378458753,
 'highSource': 'm lue1iperd tin edaec5y',
 'low': 147.6,
 'lowTime': 1642570752720,
 'lowSource': 'acdeinrdlue e pie 51myt',
 'latestPrice': 152.83,
 'latestSource': 'Close',
 'latestTime': 'July 23, 2021',
 'latestUpdate': 1691231455567,
 'latestVolume': 73044036,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 152.86,
 'delayedPriceTime': 1655265378902,
 'oddLotDelayedPrice': 149.26,
 'oddLotDelayedPriceTime': 1696507041906,
 'extendedPrice': 149.23,
 'extendedChange': -0.3,
 'extendedChangePercent': -0.00204,
 'extendedPriceTime': 1654635156825,
 'previousClose': 150.9,
 'previousVolume': 77426042,
 'change': 1.84,
 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
pe_ratio = data['peRatio']
pe_ratio

34.24

## Using Batch API Calls for good Performance

Batch API calls are one of the easiest ways to improve the performance of API calls.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give us discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this project from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,156.42,51.19,N/A
1,AAL,21.50,-2.2,N/A
2,AAP,219.29,23.14,N/A
3,AAPL,152.07,34.71,N/A
4,ABBV,121.75,40.66,N/A
...,...,...,...,...
500,YUM,128.13,33.86,N/A
501,ZBH,162.56,60.6,N/A
502,ZBRA,558.80,47.38,N/A
503,ZION,51.32,6.12,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000


We can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,702.20,5.04,28
1,ZION,51.32,6.12,389
2,CINF,118.35,6.14,168
3,AFL,54.23,6.99,368
4,UNM,27.85,7.37,718
5,WHR,228.83,7.59,87
6,HRB,25.33,8.27,789
7,SYF,48.49,8.34,412
8,COO,430.46,8.7,46
9,PGR,95.44,8.76,209


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [12]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame(rv_dataframe). 'rv' stands for `robust value'.

Note that data of EBITDA and GP of few stocks are missing, so to avoid error, we will use a try and error block.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.83,N/A,52.14,N/A,9.71,N/A,8.17,N/A,33.080709,N/A,15.312790,N/A,N/A
1,AAL,22.10,N/A,-2.24,N/A,-1.81,N/A,0.7684,N/A,-4.516548,N/A,1.912907,N/A,N/A
2,AAP,219.47,N/A,23.7,N/A,4.02,N/A,1.36,N/A,12.092045,N/A,2.950330,N/A,N/A
3,AAPL,155.86,N/A,34.75,N/A,37.06,N/A,7.76,N/A,24.794197,N/A,19.153357,N/A,N/A
4,ABBV,121.44,N/A,40.67,N/A,15.25,N/A,4.35,N/A,13.124065,N/A,8.677253,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.65,N/A,33.18,N/A,-4.66,N/A,6.52,N/A,24.646203,N/A,10.710086,N/A,N/A
501,ZBH,160.75,N/A,59.3,N/A,2.75,N/A,4.9,N/A,28.181154,N/A,8.194911,N/A,N/A
502,ZBRA,574.41,N/A,47.31,N/A,12.42,N/A,6.28,N/A,35.195379,N/A,14.229249,N/A,N/A
503,ZION,51.05,N/A,6.23,N/A,1.15,N/A,2.5,N/A,4.941484,N/A,2.449269,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

We can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.400,N/A,62.16,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,238.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,284.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.08,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.890,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.520,N/A,14.74,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.560,N/A,32.87,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,34.260,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2883.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.008,N/A,32.1,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this project, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.823762
1      0.116832
2      0.443564
3      0.643564
4      0.740594
         ...   
500    0.613861
501    0.861386
502     0.80396
503    0.130693
504    0.849505
Name: PE Percentile, Length: 505, dtype: object
0       0.788119
1      0.0752475
2       0.538614
3       0.962376
4       0.871287
         ...    
500    0.0732673
501     0.381188
502     0.837624
503     0.127723
504     0.934653
Name: PB Percentile, Length: 505, dtype: object
0       0.815842
1      0.0712871
2       0.173267
3        0.80198
4       0.577228
         ...    
500     0.742574
501     0.629703
502     0.732673
503     0.361386
504     0.918812
Name: PS Percentile, Length: 505, dtype: object
0       0.879208
1      0.0356436
2       0.318812
3       0.718812
4       0.366337
         ...    
500     0.716832
501      0.79604
502      0.89703
503    0.0574257
504     0.906931
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.821782
1      0.0772277
2       0.138614
3        0.89

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.83,N/A,52.14,0.823762,9.71,0.788119,8.1700,0.815842,33.080709,0.879208,15.312790,0.821782,N/A
1,AAL,22.10,N/A,-2.24,0.116832,-1.81,0.0752475,0.7684,0.0712871,-4.516548,0.0356436,1.912907,0.0772277,N/A
2,AAP,219.47,N/A,23.70,0.443564,4.02,0.538614,1.3600,0.173267,12.092045,0.318812,2.950330,0.138614,N/A
3,AAPL,155.86,N/A,34.75,0.643564,37.06,0.962376,7.7600,0.80198,24.794197,0.718812,19.153357,0.89505,N/A
4,ABBV,121.44,N/A,40.67,0.740594,15.25,0.871287,4.3500,0.577228,13.124065,0.366337,8.677253,0.514851,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.65,N/A,33.18,0.613861,-4.66,0.0732673,6.5200,0.742574,24.646203,0.716832,10.710086,0.667327,N/A
501,ZBH,160.75,N/A,59.30,0.861386,2.75,0.381188,4.9000,0.629703,28.181154,0.79604,8.194911,0.489109,N/A
502,ZBRA,574.41,N/A,47.31,0.80396,12.42,0.837624,6.2800,0.732673,35.195379,0.89703,14.229249,0.788119,N/A
503,ZION,51.05,N/A,6.23,0.130693,1.15,0.127723,2.5000,0.361386,4.941484,0.0574257,2.449269,0.108911,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.83,N/A,52.14,0.823762,9.71,0.788119,8.1700,0.815842,33.080709,0.879208,15.312790,0.821782,0.825743
1,AAL,22.10,N/A,-2.24,0.116832,-1.81,0.0752475,0.7684,0.0712871,-4.516548,0.0356436,1.912907,0.0772277,0.0752475
2,AAP,219.47,N/A,23.70,0.443564,4.02,0.538614,1.3600,0.173267,12.092045,0.318812,2.950330,0.138614,0.322574
3,AAPL,155.86,N/A,34.75,0.643564,37.06,0.962376,7.7600,0.80198,24.794197,0.718812,19.153357,0.89505,0.804356
4,ABBV,121.44,N/A,40.67,0.740594,15.25,0.871287,4.3500,0.577228,13.124065,0.366337,8.677253,0.514851,0.614059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.65,N/A,33.18,0.613861,-4.66,0.0732673,6.5200,0.742574,24.646203,0.716832,10.710086,0.667327,0.562772
501,ZBH,160.75,N/A,59.30,0.861386,2.75,0.381188,4.9000,0.629703,28.181154,0.79604,8.194911,0.489109,0.631485
502,ZBRA,574.41,N/A,47.31,0.80396,12.42,0.837624,6.2800,0.732673,35.195379,0.89703,14.229249,0.788119,0.811881
503,ZION,51.05,N/A,6.23,0.130693,1.15,0.127723,2.5000,0.361386,4.941484,0.0574257,2.449269,0.108911,0.157228


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()

Enter the value of your portfolio:1000000


In [23]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/home/shreyansh/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.00,714,7.37,0.134653,0.5448,0.0792079,0.4416,0.0277228,3.565213,0.0514851,0.412578,0.00990099,0.0605941
1,AIV,6.85,2919,-74.46,0.0237624,2.0100,0.29604,-31.3800,0.0019802,-41.489374,0.00792079,-51.570078,0.00594059,0.0671287
2,AIG,47.56,420,-11.14,0.0831683,0.6712,0.0831683,0.9320,0.0851485,6.077698,0.0712871,0.923741,0.0257426,0.069703
3,L,54.98,363,-399.70,0.00594059,0.8386,0.0950495,1.0228,0.10297,7.789438,0.124752,0.959965,0.0277228,0.0712871
4,MCK,206.66,96,-7.21,0.0970297,-1585.4400,0.0039604,0.1408,0.00990099,7.873177,0.128713,2.696492,0.124752,0.0728713
5,AAL,22.10,904,-2.24,0.116832,-1.8100,0.0752475,0.7684,0.0712871,-4.516548,0.0356436,1.912907,0.0772277,0.0752475
6,HFC,29.19,685,-30.21,0.049505,0.8700,0.10297,0.4144,0.0217822,7.306351,0.0950495,2.551480,0.112871,0.0764356
7,FTI,7.19,2781,9.59,0.156436,0.8628,0.10099,0.2814,0.0138614,3.258416,0.049505,3.059263,0.154455,0.0950495
8,HPQ,29.30,682,10.17,0.176238,-10.3600,0.0336634,0.5763,0.039604,7.403527,0.10099,3.081211,0.156436,0.101386
9,BA,231.16,86,-11.29,0.0811881,-7.3500,0.0653465,2.3100,0.334653,-18.588949,0.019802,-39.422650,0.00792079,0.101782


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users.

In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
Formats include colors, fonts, and also symbols like % and $. We'll need five main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* Integer format for the number of shares to purchase
* Float format with 1 decimal for each valuation metric
* Percent format for percentiles and rv_score


In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [27]:
writer.save()